In [1]:
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import pandas as pd
from datetime import datetime

# 1. Setup & Configuration
load_dotenv(override=True)
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
MODEL = 'gpt-4o-mini'

# --- MOCK DATABASE (Simulating a Real Backend) ---
# In a real app, these would be SQL tables or Google Sheets
slots_db = {
    "Morning": ["06:00", "07:00", "08:00", "09:00"],
    "Evening": ["17:00", "18:00", "19:00", "20:00", "21:00"]
}
leads_db = []  # Stores captured user info
faq_knowledge_base = {
    "parking": "Yes, we have free valet parking for 2 wheelers and paid parking for 4 wheelers.",
    "cancel": "Memberships are non-refundable, but can be transferred to a friend for a ₹500 fee.",
    "freeze": "You can freeze your annual membership for up to 30 days for medical reasons.",
    "trainers": "All our trainers are K11 or ESA certified with 3+ years of experience.",
    "ladies": "We have a dedicated ladies-only batch from 11:00 AM to 12:00 PM and 4:00 PM to 5:00 PM."
}

# --- TOOLS (The "Hands" of the AI) ---

def check_availability(time_of_day):
    """Checks available slots for Morning or Evening."""
    return json.dumps(slots_db.get(time_of_day, ["No slots"]))

def book_trial(name, phone, time, goal):
    """Saves a lead to the database."""
    # Logic: Mark as 'Hot Lead' if they have a specific goal
    status = "Hot Lead" if goal else "Warm Lead"
    
    lead_entry = {
        "Name": name,
        "Phone": phone,
        "Time": time,
        "Goal": goal,
        "Status": status,
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    leads_db.append(lead_entry)
    return f"Success! {name} is booked for {time}. Lead marked as {status}."

def generate_mini_plan(goal, current_weight):
    """Generates a quick value-add plan to impress the user."""
    # In reality, this could query a database of templates
    if "loss" in goal.lower():
        return f"Based on {current_weight}kg: Focus on a 300kcal deficit. Try High Protein (1.5g/kg). Workout: 3 days Strength + 2 days HIIT."
    elif "muscle" in goal.lower() or "gain" in goal.lower():
        return f"Based on {current_weight}kg: Focus on a 200kcal surplus. Hypertrophy training (8-12 reps). Creatine recommended."
    else:
        return "Focus on 'Recomposition'. Maintenance calories, high protein, and progressive overload."

def get_policy_answer(topic):
    """Retrieves specific policy details."""
    for key, value in faq_knowledge_base.items():
        if key in topic.lower():
            return value
    return "I'd need to check with the manager on that specific detail."

# Tool Definitions for OpenAI
tools = [
    {
        "type": "function",
        "function": {
            "name": "check_availability",
            "description": "Check available gym trial slots. Input must be 'Morning' or 'Evening'.",
            "parameters": {"type": "object", "properties": {"time_of_day": {"type": "string", "enum": ["Morning", "Evening"]}}, "required": ["time_of_day"]}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "book_trial",
            "description": "Book a trial. MANDATORY: Ask for Name, Phone, and Goal first.",
            "parameters": {
                "type": "object", 
                "properties": {
                    "name": {"type": "string"}, 
                    "phone": {"type": "string"}, 
                    "time": {"type": "string"},
                    "goal": {"type": "string"}
                }, 
                "required": ["name", "phone", "time"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_mini_plan",
            "description": "Provide a quick diet/workout summary to impress the user.",
            "parameters": {"type": "object", "properties": {"goal": {"type": "string"}, "current_weight": {"type": "number"}}, "required": ["goal", "current_weight"]}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_policy_answer",
            "description": "Get official answers for FAQs like parking, cancellation, freezing, trainers, ladies batches.",
            "parameters": {"type": "object", "properties": {"topic": {"type": "string"}}, "required": ["topic"]}
        }
    }
]

# --- CORE CHAT LOGIC ---

system_prompt = """
You are 'Coach Neo', the AI Manager at Square Fitness, Thane.
Your Goal: Sell memberships by being helpful, not pushy.

STRATEGY:
1. **Discovery**: Always ask about their fitness goal (Weight Loss/Muscle Gain) before talking price.
2. **Value First**: If they share their weight/goal, offer to generate a 'Mini Plan' using the tool. This builds trust.
3. **Objection Handling**: If they ask about parking, cancellation, etc., use the `get_policy_answer` tool.
4. **Closing**: When they seem happy, ask to book a FREE trial. Check availability first!

PRICING (Only reveal if asked):
- Silver: ₹6,000 (Gym Only)
- Gold: ₹10,000 (Gym + Group Classes + Steam)
- Platinum: ₹15,000 (All Access + 5 PT Sessions)
"""

def chat_logic(message, history):
    messages = [{"role": "system", "content": system_prompt}]
    for h in history:
        messages.append({"role": "user", "content": h['content']}) if h['role'] == 'user' else messages.append({"role": "assistant", "content": h['content']})
    messages.append({"role": "user", "content": message})

    # Step 1: Initial AI thought
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    msg = response.choices[0].message

    # Step 2: Tool Execution Loop
    if msg.tool_calls:
        messages.append(msg) # Add the intent to history
        for tool in msg.tool_calls:
            args = json.loads(tool.function.arguments)
            fname = tool.function.name
            
            # Router
            result = "Error"
            if fname == "check_availability":
                result = check_availability(args["time_of_day"])
            elif fname == "book_trial":
                result = book_trial(args["name"], args["phone"], args["time"], args.get("goal", "General"))
            elif fname == "generate_mini_plan":
                result = generate_mini_plan(args["goal"], args["current_weight"])
            elif fname == "get_policy_answer":
                result = get_policy_answer(args["topic"])
            
            messages.append({"role": "tool", "tool_call_id": tool.id, "name": fname, "content": str(result)})
        
        # Step 3: Final Response after tool usage
        final_res = client.chat.completions.create(model=MODEL, messages=messages)
        return final_res.choices[0].message.content
    
    return msg.content

# --- ADVANCED UI (Dashboard Style) ---

def get_leads_dataframe():
    """Returns the leads for the Admin view"""
    if not leads_db:
        return pd.DataFrame(columns=["Name", "Phone", "Time", "Status"])
    return pd.DataFrame(leads_db)

with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🏋️ Square Fitness Pro - AI Sales Agent")
    
    with gr.Row():
        # Left Column: The Chatbot
        with gr.Column(scale=2):
            chatbot = gr.ChatInterface(
                fn=chat_logic, 
                type="messages",
                examples=["How much is the membership?", "I want to lose weight, I am 85kg.", "Do you have parking?", "Book a trial for me."]
            )
        
        # Right Column: Admin Dashboard & Info
        with gr.Column(scale=1):
            gr.Markdown("### 📋 Live Pricing")
            gr.DataFrame(
                headers=["Tier", "Price", "Features"],
                value=[
                    ["Silver", "₹6,000", "Gym Only"],
                    ["Gold", "₹10,000", "Gym + Classes + Steam"],
                    ["Platinum", "₹15,000", "All Access + PT"]
                ]
            )
            
            gr.Markdown("---")
            gr.Markdown("### 🕵️ Admin View: Captured Leads")
            # This button refreshes the leads view
            refresh_btn = gr.Button("Refresh Leads Database")
            leads_table = gr.DataFrame(value=get_leads_dataframe())
            
            refresh_btn.click(get_leads_dataframe, outputs=leads_table)


app.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
